Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [11]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [22]:
use_mingpt = False # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'mps'

In [23]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

In [24]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [27]:
generate(prompt='How are you', num_samples=10, steps=20)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


--------------------------------------------------------------------------------
How are you doing, what's going to do you do?"

"Just come back. I'll do
--------------------------------------------------------------------------------
How are you thinking about starting?

You are thinking about starting. I would say it's a matter of
--------------------------------------------------------------------------------
How are you so proud of this guy, you need a man like him? -G.

I never
--------------------------------------------------------------------------------
How are you?

Yes. The most difficult thing I want to do is get the information out there so
--------------------------------------------------------------------------------
How are you doing after work? Are you running up and down?

Do you have to pick up the
--------------------------------------------------------------------------------
How are you guys doing here? And what do you think you should do about it?"

Tolga
------